In [1]:
import numpy as np
from PIL import Image
from pathlib import Path
from PIL import Image
import os

In [ ]:
import os
from pathlib import Path

# Get the current file's directory
current_dir = Path().absolute()

# Set working directory to parent directory (one level up)
parent_dir = current_dir.parent
os.chdir(parent_dir)

# Print confirmation
print(f"Working directory set to: {os.getcwd()}")

Working directory set to: c:\Users\alt_user\Desktop\Teknofest25Veri


In [ ]:
import shutil

# shutil.copyfile(src, dst)

# # 2nd option
# shutil.copy(src, dst) 

IMAGE_EXTENSIONS = {'.png'}

def read_images(root_dir):
    root = Path(root_dir)
    for path in root.rglob('*'):
        if path.suffix.lower() in IMAGE_EXTENSIONS:
            yield path


for path in read_images('ReadableVersion/Yarışma 2.aşama veri seti kümesi'):
    if(str(path).find("CT")!=-1):
        continue
    
    series=path.stem.split(".")[1]
    caseNum=str(path.parent).split("\\")[-3]
    # print(path)
    label=path.stem.split("_")[1]
    dst=f"Yolo/Toplu/{label}"
    os.makedirs(dst, exist_ok=True)
    name=f"{caseNum}_"+path.name
    shutil.copyfile(path, os.path.join(dst, name))
    print(f"Copied {path} to {os.path.join(dst, name)}")

Copied ReadableVersion\Yarışma 2.aşama veri seti kümesi\Vaka_300571\MR\Seri2\50009482.1.45_NormalKronik.png to Yolo/Toplu/NormalKronik\Vaka_300571_50009482.1.45_NormalKronik.png
Copied ReadableVersion\Yarışma 2.aşama veri seti kümesi\Vaka_300571\MR\Seri2\50009482.1.65_NormalKronik.png to Yolo/Toplu/NormalKronik\Vaka_300571_50009482.1.65_NormalKronik.png
Copied ReadableVersion\Yarışma 2.aşama veri seti kümesi\Vaka_300571\MR\Seri4\50009482.3.49_NormalKronik.png to Yolo/Toplu/NormalKronik\Vaka_300571_50009482.3.49_NormalKronik.png
Copied ReadableVersion\Yarışma 2.aşama veri seti kümesi\Vaka_300571\MR\Seri4\50009482.3.69_NormalKronik.png to Yolo/Toplu/NormalKronik\Vaka_300571_50009482.3.69_NormalKronik.png
Copied ReadableVersion\Yarışma 2.aşama veri seti kümesi\Vaka_300571\MR\Seri5\50009482.4.13_NormalKronik.png to Yolo/Toplu/NormalKronik\Vaka_300571_50009482.4.13_NormalKronik.png
Copied ReadableVersion\Yarışma 2.aşama veri seti kümesi\Vaka_300571\MR\Seri5\50009482.4.9_NormalKronik.png to 

In [4]:
import pandas as pd

pathA='Yolo/Toplu/HiperakutAkut'
pathK='Yolo/Toplu/NormalKronik'
pathS='Yolo/Toplu/Subakut'

if(not os.path.exists(pathA)):
    assert False, f"Path {pathA} does not exist."
if(not os.path.exists(pathK)):
    assert False, f"Path {pathK} does not exist."
if(not os.path.exists(pathS)):
    assert False, f"Path {pathS} does not exist."
    
df=pd.DataFrame(os.listdir(pathA)+os.listdir(pathK)+ os.listdir(pathS))
# Rename the original column
df = df.rename(columns={0: 'filename'})

# # Split the filenames by "_" and create new columns
df[["vaka",'case_number', 'slice_info', "label"]] = df['filename'].str.split('_', n=4, expand=True)


df

,filename,vaka,case_number,slice_info,label
0,Vaka_300663_50467107.2.16_HiperakutAkut.png,Vaka,300663,50467107.2.16,HiperakutAkut.png
1,Vaka_300663_50467107.2.19_HiperakutAkut.png,Vaka,300663,50467107.2.19,HiperakutAkut.png
2,Vaka_300663_50467107.3.19_HiperakutAkut.png,Vaka,300663,50467107.3.19,HiperakutAkut.png
3,Vaka_300663_50467107.3.21_HiperakutAkut.png,Vaka,300663,50467107.3.21,HiperakutAkut.png
4,Vaka_300663_50467107.4.0_HiperakutAkut.png,Vaka,300663,50467107.4.0,HiperakutAkut.png
...,...,...,...,...,...
4399,Vaka_380157_50451845.3.8_Subakut.png,Vaka,380157,50451845.3.8,Subakut.png
4400,Vaka_380157_50451845.3.9_Subakut.png,Vaka,380157,50451845.3.9,Subakut.png
4401,Vaka_380895_50418540.0.6_Subakut.png,Vaka,380895,50418540.0.6,Subakut.png
4402,Vaka_380895_50418540.0.7_Subakut.png,Vaka,380895,50418540.0.7,Subakut.png


In [5]:
# First split: 90% train+val, 10% test (stratified)
from sklearn.model_selection import train_test_split

X=df["slice_info"].to_numpy()
y=df["label"].to_numpy()

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Check class distribution
from collections import Counter
print("Train:", Counter(y_train))
print("Val:", Counter(y_val))
# print("Test:", Counter(y_test))

Train: Counter({'HiperakutAkut.png': 1815, 'NormalKronik.png': 1466, 'Subakut.png': 242})
Val: Counter({'HiperakutAkut.png': 454, 'NormalKronik.png': 366, 'Subakut.png': 61})


In [6]:
df[df["slice_info"]==X_train[0]]

,filename,vaka,case_number,slice_info,label
1014,Vaka_346577_50312349.3.18_HiperakutAkut.png,Vaka,346577,50312349.3.18,HiperakutAkut.png


In [7]:
df[df["slice_info"]==X_train[0]]["filename"].values[0]

'Vaka_346577_50312349.3.18_HiperakutAkut.png'

In [8]:
parent_dir="Yolo/Toplu"

for im in X_train:
    fileName=df[df["slice_info"]==im]["filename"].values[0]
    label=df[df["slice_info"]==im]["label"].values[0][:-4]
    src=os.path.join(parent_dir, label, fileName)
    os.makedirs(os.path.join(parent_dir,"train/"+ label), exist_ok=True)
    dst=os.path.join(parent_dir,"train/"+ label, fileName)
    os.rename(src, dst)


for im in X_val:
    fileName=df[df["slice_info"]==im]["filename"].values[0]
    label=df[df["slice_info"]==im]["label"].values[0][:-4]
    src=os.path.join(parent_dir, label, fileName)
    os.makedirs(os.path.join(parent_dir,"val/"+ label), exist_ok=True)
    dst=os.path.join(parent_dir,"val/"+ label, fileName)
    os.rename(src, dst)